In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql
import emoji

In [4]:
from db_util import get_db

In [4]:
db = get_db('beauty_shop')
cursor = db.cursor()

In [5]:
sql = "SHOW DATABASES"
cursor.execute(sql)
result = cursor.fetchall()
result

(('beauty_shop',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('slr',),
 ('sqldb',),
 ('sys',))

In [6]:
sql = "USE beauty_shop"
cursor.execute(sql)

0

In [7]:
# 현재 세션이 어떤 데이터베이스를 사용하고 있는 중인지 출력하는 쿼리
sql = "SELECT DATABASE()"
cursor.execute(sql)
result = cursor.fetchone()
result

('beauty_shop',)

In [8]:
# 테이블 생성
sql = '''
CREATE TABLE IF NOT EXISTS product (
PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
TITLE VARCHAR(200) NOT NULL,
ORI_PRICE VARCHAR(100),
DISCOUNT_PRICE VARCHAR(100),
link VARCHAR(200),
PRIMARY KEY(PRODUCT_CODE)
);
'''
cursor.execute(sql)
db.commit()

In [9]:
sql = "SHOW TABLES"
cursor.execute(sql)
result = cursor.fetchall()
result

(('product',),)

In [10]:
sql = "DESC product"
cursor.execute(sql)
result = cursor.fetchall()
result

(('PRODUCT_CODE', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

In [5]:
def crawling_jolse(page_num):
    try:
        print(f"Crawling page: {page_num}")
        url = f"https://jolse.com/category/toners-mists/1019?page={page_num}"
        html = urlopen(url)
        htmls = html.read()
        bs_obj = BeautifulSoup(htmls, "html.parser")

        li_list = bs_obj.select('.prdList > li')
        if not li_list:
            print("Finished crawling: No more items.")
            return False

        product_list = []
        for li in li_list:
            try:
                product_code = li.get('id').replace('anchorBoxId_', '')
                title_elements = li.select('.description > .name > a > span')
                if len(title_elements) < 2: continue
                
                title = title_elements[1].text.strip()
                title = emoji.replace_emoji(title, replace='')
                
                price_list = li.select('.description > ul > li')
                ori_price = price_list[0].select('span')[1].text.strip()
                discount_price = price_list[1].select('span')[1].text.strip()
                
                img_link = li.select_one('.prdImg > a')
                if not img_link: continue
                
                link = 'https://jolse.com' + img_link.get('href')
                link = emoji.replace_emoji(link, replace='')
                
                product = (int(product_code), title, ori_price, discount_price, link)
                product_list.append(product)
            except (AttributeError, IndexError, ValueError) as e:
                print(f"Error parsing local item: {e}")
                continue
        
        if product_list:
            print(f"Save Page: {page_num}")
            crawling_jolse_save(product_list)

        return True
    except Exception as e:
        print(f"Error crawling page {page_num}: {e}")

def crawling_jolse_save(product_list):
    db = None
    try:
        db = get_db('beauty_shop')
        cursor = db.cursor()
        sql = 'insert into product (product_code, title, ori_price, discount_price, link) values (%s, %s, %s, %s, %s)'
        cursor.executemany(sql, product_list)
        db.commit()
    except Exception as e:
        print(f"Database error: {e}")
        if db:
            db.rollback()
    finally:
        if db:
            db.close()

def crawling_start():
    page_num = 1
    while True:
        try:
            if crawling_jolse(page_num):
                page_num += 1
            else:
                break
        except ValueError:
            print("Invalid input. Please enter a valid page number.")


In [6]:
crawling_start()

Crawling page: 1
Save Page: 1
Crawling page: 2
Save Page: 2
Crawling page: 3
Save Page: 3
Crawling page: 4
Save Page: 4
Crawling page: 5
Save Page: 5
Crawling page: 6
Save Page: 6
Crawling page: 7
Save Page: 7
Crawling page: 8
Save Page: 8
Crawling page: 9
Save Page: 9
Crawling page: 10
Save Page: 10
Crawling page: 11
Save Page: 11
Crawling page: 12
Save Page: 12
Crawling page: 13
Save Page: 13
Crawling page: 14
Finished crawling: No more items.
